In [1]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr

Using TensorFlow backend.


In [21]:

pandf = fdr.DataReader("KS11","2010-01-01","2020-01-01")

In [3]:
#특성 추가 ------------------------------------------
    #이동평균선
def get_MA(df):
    MA_26=df["Close"].rolling(26).mean()
    MA_52=df["Close"].rolling(52).mean()
    df=df.assign(MA_26=MA_26,MA_52=MA_52).dropna()
    
    return df

    
    #스토캐스틱
def get_stochastic(df, n=15, m=5, t=3):
    # n일중 최고가
    ndays_high = df.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.Close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df
   
    #시간
def get_time(df):
    time=np.linspace(0,10,len(df),endpoint=False).reshape(-1,1)
    df=df.assign(time=time)
    
    return df

In [22]:
df=pandf
df=get_time(df)
df=get_stochastic(df)
df=get_MA(df)
# convert nparray



In [23]:

# day 1, 20 / 값, 이동평균선
MA_26=df["Close"].rolling(21).mean()
y_before=MA_26.dropna()

dfA=df[20:] #값과 값 -> [:] 이평선 -> [20:]  <- [20: 0 XXXX]

#y=np.where(np.array(dfA["Close"].shift(-1).dropna())>np.array(y_before.shift(-1).dropna()),1,0) #값과 이평선 ( 1일 차 )
#y=np.where(dfA["Change"].shift(-1).dropna()>0,1,0) #값과 값 ( 1일 차 )
y=np.where(np.array(dfA["Close"].shift(-20).dropna())>np.array(y_before.shift(-20).dropna()),1,0) #값과 이평선( 20일 차 )
#y=np.where(dfA["Change"].shift(-20).dropna()>0,1,0) #값과 값 ( 20일 차 )

df = dfA[:-20] # 1일차일시 [:-1] 20일차일시 [:-20]
# normalization
X=df

In [24]:
# split train, test
train_size = int(len(X) * 0.7)
y_train2=y[:train_size]
y_test2=y[train_size:]
X_train2 = X[:train_size]
X_test2 = X[train_size:]
X_train2.shape


(1663, 12)

In [25]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train2, (X_train2.shape[0], 1, X_train2.shape[1]))
X_test = np.reshape(X_test2, (X_test2.shape[0], 1, X_test2.shape[1]))
y_train=y_train2
y_test=y_test2


ValueError: Must pass 2-d input

In [9]:
# simple lstm network learning
model = Sequential()
model.add(LSTM(36, input_shape=(1, 12)))
for i in range(3):
    model.add(Dense(36,activation='tanh'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, batch_size=16,validation_split=0.1)
model.evaluate(X_test,y_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1496 samples, validate on 167 samples
Epoch 1/100
1496/1496 [==============================] - 1s 391us/step - loss: 0.6886 - acc: 0.5541 - val_loss: 0.6621 - val_acc: 0.6407
Epoch 2/100
1496/1496 [==============================] - 0s 99us/step - loss: 0.6712 - acc: 0.5969 - val_loss: 0.6393 - val_acc: 0.6707
Epoch 3/100
1496/1496 [==============================] - 0s 98us/step - loss: 0.6543 - acc: 0.6263 - val_loss: 0.7260 - val_acc: 0.4072
Epoch 4/100
1496/1496 [==============================] - 0s 106us/step - loss: 0.6524 - acc: 0.6223 - val_loss: 0.7180 - val_acc: 0.4192
Epoch 5/100
1496/1496 [==============================] - 0s 97us/step - loss: 0.6460 - acc: 0.6344 - val_loss: 0.7026 - val_acc: 0.4491
Epoch 6/100
1496/1496 [==============================] - 0s 103us/step - loss: 0.6450 - acc: 0.

Epoch 54/100
1496/1496 [==============================] - 0s 111us/step - loss: 0.6231 - acc: 0.6638 - val_loss: 0.6738 - val_acc: 0.5509
Epoch 55/100
1496/1496 [==============================] - 0s 111us/step - loss: 0.6199 - acc: 0.6691 - val_loss: 0.6546 - val_acc: 0.5868
Epoch 56/100
1496/1496 [==============================] - 0s 121us/step - loss: 0.6204 - acc: 0.6711 - val_loss: 0.6608 - val_acc: 0.5629
Epoch 57/100
1496/1496 [==============================] - ETA: 0s - loss: 0.6238 - acc: 0.653 - 0s 126us/step - loss: 0.6213 - acc: 0.6544 - val_loss: 0.7222 - val_acc: 0.4251
Epoch 58/100
1496/1496 [==============================] - 0s 107us/step - loss: 0.6204 - acc: 0.6671 - val_loss: 0.6996 - val_acc: 0.4790
Epoch 59/100
1496/1496 [==============================] - 0s 114us/step - loss: 0.6202 - acc: 0.6644 - val_loss: 0.6216 - val_acc: 0.7066
Epoch 60/100
1496/1496 [==============================] - 0s 103us/step - loss: 0.6219 - acc: 0.6638 - val_loss: 0.7483 - val_acc: 0.3

[0.8006417717873465, 0.4754558205604553]

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,np.where(model.predict(X_test)>0.5,1,0))

0.47545582047685836

In [13]:
y_list=model.predict(X_test)

In [43]:
X_test2
corrMatt =X_test2.corr()
print(corrMatt)


           Close      Open      High       Low    Volume    Change      time  \
Close   1.000000  0.996446  0.998221  0.998874 -0.191077  0.044403 -0.478836   
Open    0.996446  1.000000  0.998831  0.998275 -0.179547 -0.020743 -0.474610   
High    0.998221  0.998831  1.000000  0.998460 -0.180676  0.002485 -0.472235   
Low     0.998874  0.998275  0.998460  1.000000 -0.188229  0.020353 -0.481540   
Volume -0.191077 -0.179547 -0.180676 -0.188229  1.000000 -0.140412  0.334251   
Change  0.044403 -0.020743  0.002485  0.020353 -0.140412  1.000000 -0.053309   
time   -0.478836 -0.474610 -0.472235 -0.481540  0.334251 -0.053309  1.000000   
kdj_k   0.152060  0.120238  0.124309  0.146136 -0.045018  0.454928 -0.165760   
kdj_d   0.180799  0.166844  0.163661  0.182094  0.010695  0.198027 -0.187602   
kdj_j   0.187990  0.179779  0.174726  0.191161  0.029316  0.114442 -0.193374   
MA_26   0.947727  0.951487  0.953706  0.946963 -0.182765 -0.065248 -0.424092   
MA_52   0.904037  0.906769  0.910609  0.

In [15]:
y_list.shape

(713, 1)

In [395]:
MA_26=df["Close"].rolling(26).mean()
MA_26=MA_26.dropna()
MA_26.shape


(713,)

In [396]:
pandf.shape

(789, 6)

In [397]:
MA_26.shape

(713,)

In [398]:
X_train2.shape

(516, 12)

#Feature_X

NameError: name 'train' is not defined